Setup

In [ ]:
import pandas as pd
import numpy as np
import missingno as mno # needed to visualize missing values. install missingno into conda if import does not work!
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import time
import datetime
import warnings
import pyodbc # used for connection to mssql server where data is stored
import vegafusion as vf # also needs module vl-convert-python
from sklearn.model_selection import train_test_split

vf.enable()
warnings.simplefilter(action='ignore', category=FutureWarning)
alt.data_transformers.disable_max_rows()

Read Data

In [ ]:
# establish the connection to mssql server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=rdb.hdm-server.eu;'
                      'DATABASE=W22S101;'
                      'UID=W22S101;'
                      'PWD=81305')

# create a cursor from the connection
cursor = conn.cursor()

# execute a SQL query
query = "SELECT * FROM FinTable"
df_sql = pd.read_sql(query, conn)

df_sqlartists=pd.read_sql("select * from artists",conn)
# close the connection
conn.close()

In [ ]:
df_sql.popularity=df_sql.popularity.astype('int64')

In [ ]:
df_sql

In [ ]:
df_sqlartists

In [ ]:
out = df_sql.join(pd.DataFrame(df_sql['id_artists'].str.findall(',').to_list(), 
                           index=df_sql.index)
              .reindex(columns=range(5))
              .rename(columns=lambda x: f'it{x+1}')
              )

In [ ]:
out

# Data Structure and preparation

Prepare Dataset

In [ ]:
# Write first entry of artists back into the column 
# hoping that the first entry is always the main artist of the song found in the top song dataset
df_sql['artists'] = df_sql.artists.apply(lambda x: eval(x)[0])
# Add column to identify top songs and label with 1 if position <= 50
df_sql['top_song'] = 0
df_sql['top_song'][df_sql.popularity >= 85] = 1
# change column to category type
# df_sql['top_song'] = df_sql['top_song'].astype('int')
# change time signature to category
df_sql['time_signature'] = df_sql['time_signature'].astype('int')
df_sql['time_signature'] = df_sql['time_signature'].astype('category')
# change mode to category
df_sql['mode'] = df_sql['mode'].astype('int64')
df_sql['mode'] = df_sql['mode'].astype('category')
# change explicit to category
df_sql['explicit'] = df_sql['explicit'].astype('int64')
df_sql['explicit'] = df_sql['explicit'].astype('category')
# change some feature to int
df_sql['popularity'] = df_sql['popularity'].astype('int64')
df_sql['duration_ms'] = df_sql['duration_ms'].astype('int64')
# change somex features to float
df_sql['danceability'] = df_sql['danceability'].astype('float')
df_sql['energy'] = df_sql['energy'].astype('float')
df_sql['key'] = df_sql['key'].astype('float')
df_sql['loudness'] = df_sql['loudness'].astype('float')
df_sql['speechiness'] = df_sql['speechiness'].astype('float')
df_sql['acousticness'] = df_sql['acousticness'].astype('float')
df_sql['valence'] = df_sql['valence'].astype('float')
df_sql['instrumentalness'] = df_sql['instrumentalness'].astype('float')
df_sql['liveness'] = df_sql['liveness'].astype('float')
df_sql['tempo'] = df_sql['tempo'].astype('float')
# change track number and disk number to category
# drop duplicate columns
df_sql = df_sql.drop(['ChartsArtists','ChartsTrackId','ChartsName','Chartsduration','id_artists','release_date'], axis = 1)
# change remaining object types to category
for col in df_sql.columns:
    if df_sql[col].dtype == object:
        df_sql[col] = df_sql[col].astype('category')
# find duplicate songs by the same song id and remove them from the dataframe
duplicateRows = df_sql[df_sql.duplicated(['id'])]
df_sql = df_sql.drop(duplicateRows.index)

In [ ]:
df_sql['top_song'].value_counts()

In [ ]:
# show values of mode column
df_sql['mode'].value_counts()

In [ ]:
# show values of time signature column
df_sql['time_signature'].value_counts()

In [ ]:
pd.options.display.max_rows = None
df_sql['ChartsCountry'].value_counts()


Variable list and data splitting

In [ ]:
# define outcome variable as y_label
y_label = 'top_song'
# select features
features = df_sql.drop(columns=[y_label]).columns.tolist()
# create feature data for data splitting
X = df_sql[features]
# create response for data splitting
y = df_sql[y_label]
# Data Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# data training set
df_train = pd.DataFrame(X_train.copy())
df_train = df_train.join(pd.DataFrame(y_train))

# Data Exploration

In [ ]:
df_sql.info()

In [ ]:
df_sql.head()

In [ ]:
df_explore = df_train.copy()

list_num = df_explore.select_dtypes(include=[np.number]).columns.to_list()

Descriptive Statistics

In [ ]:
df_explore.describe().round(2).T

In [ ]:
q1 = df_explore.quantile(q = 0.25)
q3 = df_explore.quantile(q = 0.75)
iqr = q3-q1
iqr

In [ ]:
# inspect correlation   
corr = df_explore.corr()
corr.style.background_gradient(cmap='Blues')

In [ ]:
# inspect correlation for first country 
filter_df = df_explore[df_explore['ChartsCountry'] == 'tr'] 
filter_df.corrwith(filter_df['top_song'])

In [ ]:
# Make a correlation with top song for every country
#def print_corr(df, country):
#    filter_df = df[df['ChartsCountry'] == country] 
#    result = filter_df.corrwith(filter_df['top_song'])
#    print(f"Correlation for {country}: \n{result}")

#list_ctry = df_explore['ChartsCountry'].to_list()

#for country in list_ctry:
#    print_corr(df_explore, country)

# Dont Run this without filtering the countries first

In [ ]:
 # generates alot of output
 # Make a correlation matrix for every country
from IPython.display import display, HTML, Markdown
def print_corr_matrix(df, country):
    filter_df = df[df['ChartsCountry'] == country] 
    corr = filter_df.corr()
    corr_display = corr.style.background_gradient(cmap='Blues')._repr_html_()
    display(HTML(corr_display))
    display(Markdown(f"Correlation matrix for {country}"))

for country in list_ctry:
    print_corr_matrix(df_explore, country)


Visual analysis

In [ ]:
# Make a boxplot for every numerical variable in relation to top song label
def print_kde(df, x_var):
    #df = df[(df['streams'] < 2000000)]


    ax = sns.scatterplot(data=df, 
            x=x_var,hue="top_song",y="popularity"
            )
    ax.set_title('Density Plot')
    ax.set_xlabel(x_var)
    plt.suptitle('')

for col in list_num:
    print_kde(df_explore, col)
    plt.show()


Top song classification analysis

In [ ]:
# Stacked bar plots for categories mode and explicit in relation to top songs
mode = alt.Chart(df_explore).mark_bar().encode(
    x=alt.X('count(mode)', stack="normalize"),
    y='top_song',
    color='mode'
)
mode.title = 'Percentage of mode per top song'

explicit = alt.Chart(df_explore).mark_bar().encode(
    x=alt.X('count(explicit)', stack="normalize"),
    y='top_song',
    color='explicit'
)
explicit.title = 'Percentage of explicit per top song'

# country = alt.Chart(df_explore).mark_bar().encode(
#     x=alt.X('count(ChartsCountry)', stack="normalize"),
#     y='top_song',
#     color='ChartsCountry'
# )
# country.title = 'Percentage top song per country'

chart = alt.vconcat(mode,
                    explicit,
                    # country
                    )

chart

In [ ]:
# Make a boxplot for every numerical variable in relation to top song label
def print_boxplot(df, y_var):
    df = df[(df['instrumentalness'] < 0.2)]
    
    ax = df.boxplot(column=y_var, by='top_song', showfliers=False)
    ax.set_title('Relation to top song label')
    ax.set_xlabel('top song')
    ax.set_ylabel(y_var)
    plt.suptitle('')

list_num.append('time_signature')

for col in list_num:
    print_boxplot(df_explore, col)
    plt.show()


First Results

* remove columns with ids for model
* keep either year or release date for model
* standard scale for numerical values 
* no outliers (tempo needs to be checked more because values of 0 dont make sense)
* Energy and loudness have a strong positive correlation - drop one?
* Acousticness and energy as well as loudness correlate moderate to stron negatively - drop one / both?
* mode more 0 for top songs, relatively low difference (~10%)
* explicit more 1 for top songs, relatively low difference (~2%)
* danceability, loudness, duration_ms could have impact on top song
* interpret difference box plot year, track number - why is it this different?
